# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Dependencies](#dependencies)**
2. **[Tables](#tables)**
2. **[Views](#views)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/utils/conversions.ipynb",
    ]
}
```

# **TABLES**

---


### DESTINATIONS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSDEST_RIVM_VACCINATIONS_PARTLY_FULLY]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSDEST_RIVM_VACCINATIONS_PARTLY_FULLY]
  START WITH 1
  INCREMENT BY 1;
GO

IF NOT EXISTS (SELECT * FROM SYS.TABLES WHERE [NAME] = 'RIVM_VACCINATIONS_PARTLY_FULLY')
CREATE TABLE [VWSDEST].[RIVM_VACCINATIONS_PARTLY_FULLY](
    [ID] [INT] PRIMARY KEY NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSDEST_RIVM_VACCINATIONS_PARTLY_FULLY]),
    [DATE_LAST_INSERTED]  [DATETIME] DEFAULT GETDATE(),
    [WEEK_START] [DATETIME] NULL,
    [WEEK_END] [DATETIME] NULL,
    [PARTIALLY_VACCINATED] [INT] NULL,
    [FULLY_VACCINATED] [INT] NULL,
    [PARTIALLY_OR_FULLY_VACCINATED] [INT] NULL,
    [BOOSTER1] [INT] NULL, 
    [BOOSTER1_INCREMENT] [INT] NULL
);
GO

IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'IX_DEST_RIVM_VACCINATIONS_PARTLY_FULLY_DLI')
CREATE NONCLUSTERED INDEX [IX_DEST_RIVM_VACCINATIONS_PARTLY_FULLY_DLI]
ON [VWSDEST].[RIVM_VACCINATIONS_PARTLY_FULLY] ([DATE_LAST_INSERTED])
INCLUDE (WEEK_START);
GO

# **VIEWS**

---


In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER VIEW VWSDEST.V_VACCINATIONS_PARTLY_FULLY
AS
    WITH BASE_CTE AS (
        SELECT
            [PARTIALLY_VACCINATED],
            [FULLY_VACCINATED],
            [PARTIALLY_OR_FULLY_VACCINATED],
            [BOOSTER1] AS [BOOSTER_VACCINATED],
            [DBO].[CONVERT_DATETIME_TO_UNIX](WEEK_START) AS [DATE_START_UNIX],
            [DBO].[CONVERT_DATETIME_TO_UNIX](WEEK_END) AS [DATE_END_UNIX],
            [DBO].[CONVERT_DATETIME_TO_UNIX](DATE_LAST_INSERTED) AS [DATE_OF_INSERTION_UNIX]
        FROM [VWSDEST].[RIVM_VACCINATIONS_PARTLY_FULLY]
        WHERE DATE_LAST_INSERTED = (SELECT MAX(DATE_LAST_INSERTED) FROM [VWSDEST].[RIVM_VACCINATIONS_PARTLY_FULLY])
            AND WEEK_START >= '2021-01-04 00:00:00.000' -- VACCINATION STARTED IN THE WEEK OF 4-1-2021
    )

    -- FILTER OUT:
    -- 1. ANY INCOMPLETE WEEKS / FUTURE WEEKS
    SELECT
        [PARTIALLY_VACCINATED],
        [FULLY_VACCINATED],
        [PARTIALLY_OR_FULLY_VACCINATED],
        [BOOSTER_VACCINATED],
        [DATE_START_UNIX],
        [DATE_END_UNIX],
        [DATE_OF_INSERTION_UNIX]
    FROM BASE_CTE
    WHERE [DATE_OF_INSERTION_UNIX] > [DATE_END_UNIX];
GO

# **DATATINO CONFIGURATIONS**

---


### WORKFLOWS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN   VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

-- 1) UPSERT WORKFLOW(S).....
DECLARE @workflow_name NVARCHAR(50) = 'RIVM_VACCINATIONS_PARTLY_FULLY',
        @workflow_id INT,        
        @workflow_description VARCHAR(256),
        @is_active INT;

-- 1.1) SET ENVIRONMENTAL VARIABLES.....
SET @is_active = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 1
    WHEN 'acceptance' THEN 1
    ELSE 1
END;

SELECT TOP(1)
    @workflow_id = workflows.[ID]
FROM [DATATINO_ORCHESTRATOR_1].[WORKFLOWS] workflows
INNER JOIN [DATATINO_ORCHESTRATOR_1].[V_WORKFLOWS] v_workflows ON v_workflows.[DATAFLOW_ID] = workflows.[DATAFLOW_ID] AND v_workflows.[ID] = workflows.[ID]
WHERE v_workflows.[NAME] = @workflow_name;

SET @workflow_description = CONCAT('WORKFLOW: ', @workflow_name);

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_WORKFLOW]
    @id = @workflow_id, 
    @workflow_name = @workflow_name,
    @description = @workflow_description,
    @schedule = '0 13 * * TUE', -- At 01:00 PM, only on Tuesday
    @active = @is_active;

-- 2) UPSERT SOURCE(S).....
DECLARE @source NVARCHAR(256),
        @source_id INT,
        @source_name NVARCHAR(256),
        @source_description NVARCHAR(256),
        @target_name VARCHAR(256),
        @location_type VARCHAR(50),
        @security_profile VARCHAR(50);

-- 2.1) SET ENVIRONMENTAL VARIABLES.....
SET @source = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'https://data.rivm.nl/data/vws/covid-19/VWS_COVID-19_Gevaccineerde_mensen_12PLUS.csv'
    WHEN 'acceptance' THEN 'https://acceptatie.data.rivm.nl/data/vws/covid-19/VWS_COVID-19_Gevaccineerde_mensen_12PLUS.csv'
    ELSE 'datafiles/RIVM/VWS_COVID-19_Gevaccineerde_mensen_12PLUS.csv'
END;

SET @location_type = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'Web'
    WHEN 'acceptance' THEN 'Web'
    ELSE 'AzureBlob'
END;

SET @security_profile = CASE LOWER('#{ Environment }#')
    WHEN 'production' THEN 'RIVM'
    WHEN 'acceptance' THEN 'RIVM'
    ELSE 'RIVM'
END;

-- 2.2) UPSERT TABLE SOURCE(S): STAGING......
SET @source_name = 'SOURCE_Load_Vaccinations_Partly_Fully';
SET @source_description = CONCAT('LOAD: ', @workflow_name);
SET @target_name = CONCAT('VWSSTAGE.', @workflow_name);

SELECT TOP(1)
    @source_id = [ID]
FROM [DATATINO_ORCHESTRATOR_1].[SOURCES]
WHERE [NAME] = @source_name;

EXECUTE [DATATINO_ORCHESTRATOR_1].[UPSERT_SOURCE]
    @id = @source_id,
    @source_name = @source_name,
    @description = @source_description,
    @source = @source,
    @source_columns = 'RecordId|Status|Date_of_report|Date_of_statistics|End_iso_week|Birth_year|CumSum_vaccination_partly|CumSum_vaccination_completed|CumSum_vaccination_booster1|Population',
    @target_columns = 'RECORDID|STATUS|DATE_OF_REPORT|DATE_OF_STATISTICS|END_ISO_WEEK|BIRTH_YEAR|CUMSUM_VACCINATION_PARTLY|CUMSUM_VACCINATION_COMPLETED|CUMSUM_VACCINATION_BOOSTER|POPULATION',
    @target_name = @target_name,
    @source_type = 'CsvFile',
    @location_type = @location_type,
    @delimiter_type = 'SemiColon',
    @security_profile= @security_profile;
GO